In [2]:
import pandas as pd
import seaborn as sns
print(f"Using pandas version number: {pd.__version__}")

Using pandas version number: 1.4.2


In [3]:
from sklearn.feature_extraction import DictVectorizer

# **Solution to the homework 1**: 

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records"

## **Q1 Reading the data:**

In [4]:
data_january = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
data_january.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

We have 19 columns in this January dataset:

Let's check a prelliminary data sample:

In [3]:
data_january.sample(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
866304,2,2023-01-10 20:43:34,2023-01-10 20:50:51,2.0,1.36,1.0,N,48,143,1,9.3,1.00,0.5,1.43,0.00,1.0,15.73,2.5,0.00
2079778,1,2023-01-22 17:41:28,2023-01-22 17:49:29,2.0,1.30,1.0,N,237,170,2,10.0,2.50,0.5,0.00,0.00,1.0,14.00,2.5,0.00
524762,2,2023-01-07 02:03:04,2023-01-07 02:18:11,1.0,4.52,1.0,N,137,87,1,22.6,1.00,0.5,5.52,0.00,1.0,33.12,2.5,0.00
873375,2,2023-01-10 21:32:28,2023-01-10 21:46:18,1.0,2.07,1.0,N,143,140,1,14.9,1.00,0.5,2.10,0.00,1.0,22.00,2.5,0.00
1253079,2,2023-01-14 14:12:47,2023-01-14 14:32:45,2.0,2.58,1.0,N,246,163,1,19.1,0.00,0.5,4.62,0.00,1.0,27.72,2.5,0.00
1774134,2,2023-01-19 19:01:36,2023-01-19 19:11:56,2.0,1.51,1.0,N,142,239,1,11.4,2.50,0.5,1.00,0.00,1.0,18.90,2.5,0.00
1063099,1,2023-01-12 18:58:49,2023-01-12 19:08:08,1.0,0.60,1.0,N,236,263,1,7.2,5.00,0.5,2.75,0.00,1.0,16.45,2.5,0.00
2942979,1,2023-01-31 15:32:07,2023-01-31 16:12:38,2.0,2.30,1.0,N,262,142,1,24.7,2.50,0.5,5.74,0.00,1.0,34.44,2.5,0.00
1419784,1,2023-01-16 09:06:48,2023-01-16 09:51:58,1.0,24.80,1.0,N,138,138,1,93.3,8.75,0.5,22.00,6.55,1.0,132.10,2.5,1.25
573012,2,2023-01-07 16:50:47,2023-01-07 17:08:57,1.0,1.61,1.0,N,68,137,1,16.3,0.00,0.5,3.00,0.00,1.0,23.30,2.5,0.00


## **Q2. Computing duration:**

In [5]:
data_january['duration_mins'] = data_january.tpep_dropoff_datetime - data_january.tpep_pickup_datetime
data_january.duration_mins = data_january.duration_mins.dt.total_seconds() / 60

data_january.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration_mins
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [6]:
january_std = data_january.duration_mins.std()
print(f"the standard deviation of the trips duration in January id: {round(january_std, 2)}")

the standard deviation of the trips duration in January id: 42.59


## **Q3. Dropping outliers**

Let's remove the outliers:

In [7]:
data_january_filtered = data_january[(data_january.duration_mins >= 1) & (data_january.duration_mins <= 60)]
num_records, _ = data_january_filtered.shape
print(f"Number of records after the filtering: {num_records}")

Number of records after the filtering: 3009173


In [8]:
num_records_orig, _ = data_january.shape
left_percent = num_records / num_records_orig * 100

print(f"fraction of the records left after dropping the outliers: {round(left_percent)}%")

fraction of the records left after dropping the outliers: 98%


## **Q4. One-hot encoding:**

Apply one-hot encoding to the pickup and dropoff location IDs

In [9]:
categorical = ['PULocationID', 'DOLocationID']
target = ['duration_mins']

# Change the data type of the IDs
data_january_filtered[categorical] = data_january_filtered[categorical].astype(str)

# Create dicts
train_dicts = data_january_filtered[categorical].to_dict(orient='records')
train_dicts[:5]


/tmp/ipykernel_52963/1082704510.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_january_filtered[categorical] = data_january_filtered[categorical].astype(str)


[{'PULocationID': '161', 'DOLocationID': '141'},
 {'PULocationID': '43', 'DOLocationID': '237'},
 {'PULocationID': '48', 'DOLocationID': '238'},
 {'PULocationID': '138', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '79'}]

In [10]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [13]:
print(f"Dimensionality of the X_train matrix: {X_train.shape[1]}")

Dimensionality of the X_train matrix: 515


## **Q5. Training a model:**

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Fitting a linear regression model with default parameters:

In [14]:
y_train = data_january_filtered[target].values

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

LinearRegression()

RMSE on train

In [15]:
y_pred = lr_model.predict(X_train)
error_train = mean_squared_error(y_train, y_pred, squared=False)

print(f"RMSE on the train dataset: {round(error_train, 2)}")

RMSE on the train dataset: 7.65


## **Q6. Evaluating the model:**

validation dataset (February 2023).

In [16]:
val_data = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

In [17]:
val_data['duration_mins'] = val_data.tpep_dropoff_datetime - val_data.tpep_pickup_datetime
val_data.duration_mins = val_data.duration_mins.dt.total_seconds() / 60
val_data = val_data[(val_data.duration_mins >= 1) & (val_data.duration_mins <= 60)]

val_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration_mins
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.00,0.0,1.0,9.40,2.5,0.00,1.683333
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.00,0.0,1.0,74.65,0.0,1.25,32.083333
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.30,0.0,1.0,25.30,2.5,0.00,13.300000
5,1,2023-02-01 00:52:40,2023-02-01 01:07:18,1.0,5.10,1.0,N,148,236,1,21.9,3.50,0.5,5.35,0.0,1.0,32.25,2.5,0.00,14.633333
6,1,2023-02-01 00:12:39,2023-02-01 00:40:36,1.0,8.90,1.0,N,137,244,1,41.5,3.50,0.5,3.50,0.0,1.0,50.00,2.5,0.00,27.950000


In [18]:
val_data[categorical] = val_data[categorical].astype(str)
val_dicts = val_data[categorical].to_dict(orient='records')
val_dicts[:5]

[{'PULocationID': '142', 'DOLocationID': '163'},
 {'PULocationID': '132', 'DOLocationID': '26'},
 {'PULocationID': '161', 'DOLocationID': '145'},
 {'PULocationID': '148', 'DOLocationID': '236'},
 {'PULocationID': '137', 'DOLocationID': '244'}]

In [19]:
X_val = dv.transform(val_dicts)

In [20]:
y_val = val_data[target].values

In [21]:
y_preds = lr_model.predict(X_val)
error_val = mean_squared_error(y_val, y_preds, squared=False)

print(f"RMSE on the train dataset: {round(error_val, 2)}")

RMSE on the train dataset: 7.81
